In [ ]:
import time
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs, convert_output

physical_devices = tf.config.experimental.list_physical_devices('GPU')
num_classes = 1
tfrecord = None
weights = 'face-yolov3-tiny.tf'
classes = './yolov3_tf2/face.names'
size = 416

In [ ]:
yolo = YoloV3Tiny(classes=num_classes)
yolo.load_weights(weights).expect_partial()

In [ ]:
class_names = [c.strip() for c in open(classes).readlines()]

In [ ]:
def face_detection(img_raw):
    img_raw = tf.convert_to_tensor(img_raw, dtype=tf.uint8)
    img = tf.expand_dims(img_raw, 0)
    img = transform_images(img, size)
    boxes, scores, classes, nums = yolo(img)
    img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
    output = convert_output(img, (boxes, scores, classes, nums))
    return output

In [ ]:
from matplotlib import pyplot as plt
img_raw = cv2.imread("1.jpg")[:, :, ::-1]
plt.imshow(img_raw)
plt.show()
t1 = time.time()
face_detection(img_raw)
t2 = time.time()
print(t2-t1)
out = face_detection(img_raw)
top, right, bottom, left = out[0]
face = img_raw[top:bottom, left:right]
plt.imshow(face)
plt.show()


In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

In [ ]:
batch_size = 512
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
t1 = time.time()
x = model.predict(x_train[:10000])
t2 = time.time()
print(t2-t1)